In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras import callbacks
import keras
import keras_tuner
import keras.utils as image
from keras import layers
from keras import ops
from keras import callbacks
from keras import regularizers
from tensorflow.keras.callbacks import LearningRateScheduler

## Importing Data from csv files


In [2]:
images = pd.read_csv("/Users/sofie/Desktop/Projects/Classification of Birds/CUB_200_2011/images.txt", sep=r'\s+', names=['image_id', 'image_name'], engine='python')
train_test_split = pd.read_csv("/Users/sofie/Desktop/Projects/Classification of Birds/CUB_200_2011/train_test_split.txt", sep=r'\s+', names=['image_id', 'is_training_image'], engine='python')
classes =pd.read_csv("/Users/sofie/Desktop/Projects/Classification of Birds/CUB_200_2011/classes.txt", sep=r'\s+', names=['class_id', 'class_name'], engine='python')
image_class_labels =pd.read_csv("/Users/sofie/Desktop/Projects/Classification of Birds/CUB_200_2011/image_class_labels.txt", sep=r'\s+', names=['image_id', 'class_id'], engine='python')

## Preprocessing

In [3]:
# Merge dfs based on column names so we have one df with all the necessary info contained per each row
image_data = pd.merge(images,train_test_split, on='image_id')
image_data = pd.merge(image_data,image_class_labels, on='image_id')
image_data = pd.merge(image_data,classes, on='class_id')

In [4]:
# Split training and testing image data
training_image_data = image_data[image_data['is_training_image']==1]
testing_image_data = image_data[image_data['is_training_image']==0]

# Shuffle training data
training_image_data = training_image_data.sample(frac=1)

# Initiate empty lists for training and testing images
training_images = []
testing_images = []

# Add training and testing images to corresponding lists
for i in (training_image_data['image_name'].values):
    training_images.append(image.load_img('/Users/sofie/Desktop/Projects/Classification of Birds/CUB_200_2011/images/{}'.format(i), target_size=(224, 224)))

for i in (testing_image_data['image_name'].values):
    testing_images.append(image.load_img('/Users/sofie/Desktop/Projects/Classification of Birds/CUB_200_2011/images/{}'.format(i), target_size=(224, 224)))

# Extract class labels for training and testing images
training_class_label = np.array(training_image_data['class_id'].values)
testing_class_label = np.array(testing_image_data['class_id'].values)

In [5]:
# Convert list of images to NumPy array
training_images = np.array(training_images)
testing_images = np.array(testing_images)

# Apply preprocessing
preprocessed_training_images = preprocess_input(training_images)
preprocessed_testing_images = preprocess_input(testing_images)

We begin training by keeping the existing layers frozen, allowing only the newly added classification head to learn. This ensures that the output layer is properly trained before fine-tuning the deeper layers. Once the classifier stabilizes, we gradually unfreeze and fine-tune the earlier layers, optimizing them in a controlled manner. This approach prevents instability in feature extraction and allows each layer to adjust effectively, improving overall model performance.

In [6]:
# Load pre-trained Xception model (without top layers)
base_model = keras.applications.xception.Xception(weights="imagenet", include_top=False)

# Add new layers on top
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
# dropout1 = keras.layers.Dropout(0.4, noise_shape=None, seed=None)(avg) 
dense = keras.layers.Dense(512, activation="relu", kernel_regularizer=regularizers.l2(l2=1e-4))(avg)  # L2 regularization
# dropout2 = keras.layers.Dropout(0.4, noise_shape=None, seed=None)(dense) 
output = keras.layers.Dense(201, activation="softmax")(dense)  # 201 classes

# Create model
model = keras.Model(inputs=base_model.input, outputs=output)

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Define optimizer
optimizer = keras.optimizers.Adam()  # Use Adam with a fixed learning rate

# Compile model
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

# Define function to reduce LR at epoch 20
# def scheduler(epoch, lr):
#     if epoch == 20:  # If validation accuracy is plateauing, reduce LR
#         return lr * 0.1  # Reduce LR by 10x
#     return lr  # Otherwise, keep it the same

# Create LearningRateScheduler callback
# lr_callback = LearningRateScheduler(scheduler)

# Set early stopping
earlystopping = callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=3, restore_best_weights=True)

# Define validation data split at 30%
split_idx = int(len(preprocessed_training_images) * 0.7)
X_train, X_val = preprocessed_training_images[:split_idx], preprocessed_training_images[split_idx:]
y_train, y_val = training_class_label[:split_idx], training_class_label[split_idx:]

y_train = np.array(y_train).astype(np.int32)
y_val = np.array(y_val).astype(np.int32)

# Train model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), batch_size=32, callbacks=[earlystopping])

Epoch 1/50
132/132 ━━━━━━━━━━━━━━━━━━━━ 137s 1s/step - accuracy: 0.0097 - loss: 5.4586 - val_accuracy: 0.0767 - val_loss: 5.1009 - learning_rate: 1.0000e-04
Epoch 2/50
132/132 ━━━━━━━━━━━━━━━━━━━━ 135s 1s/step - accuracy: 0.0500 - loss: 5.0546 - val_accuracy: 0.1523 - val_loss: 4.6752 - learning_rate: 1.0000e-04
Epoch 3/50
132/132 ━━━━━━━━━━━━━━━━━━━━ 135s 1s/step - accuracy: 0.1038 - loss: 4.6068 - val_accuracy: 0.2046 - val_loss: 4.1398 - learning_rate: 1.0000e-04
Epoch 4/50
132/132 ━━━━━━━━━━━━━━━━━━━━ 136s 1s/step - accuracy: 0.1456 - loss: 4.1412 - val_accuracy: 0.2546 - val_loss: 3.6894 - learning_rate: 1.0000e-04
Epoch 5/50
132/132 ━━━━━━━━━━━━━━━━━━━━ 136s 1s/step - accuracy: 0.1986 - loss: 3.7553 - val_accuracy: 0.2974 - val_loss: 3.3721 - learning_rate: 1.0000e-04
Epoch 6/50
132/132 ━━━━━━━━━━━━━━━━━━━━ 137s 1s/step - accuracy: 0.2386 - loss: 3.4329 - val_accuracy: 0.3335 - val_loss: 3.1344 - learning_rate: 1.0000e-04
Epoch 7/50
132/132 ━━━━━━━━━━━━━━━━━━━━ 138s 1s/step - acc

KeyboardInterrupt: 

In [7]:
model.evaluate(preprocessed_testing_images, testing_class_label)

 10/182 ━━━━━━━━━━━━━━━━━━━━ 2:04 723ms/step - accuracy: 0.6645 - loss: 1.4523

KeyboardInterrupt: 